In [62]:
import pandas as pd
import numpy as np
#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer


In [63]:
df = pd.read_csv('/Users/noahpovis/Desktop/Buildweek22/Build Week 2 /Buildweek22.csv', mangle_dupe_cols=True)


In [64]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [65]:
train.shape

(16278, 19)

In [66]:
val.shape

(5426, 19)

In [67]:
test.shape

(5427, 19)

In [68]:
def clean(i):
    i = i.copy()
    i.drop_duplicates(subset=['Organization Name','Funding Type'], inplace = True)
    i.drop_duplicates(keep='first', inplace= True)
    i['Raised Series A'] = np.where(i['Organization Name'].duplicated(), 'Yes', 'No')
    i.drop_duplicates(subset='Organization Name', keep='last', inplace= True)
    i['Money Raised Currency (in USD)'] = pd.to_numeric(i['Money Raised Currency (in USD)'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i['Announced Date'] = pd.to_datetime(i['Announced Date'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i  = i[i['Funding Type'] != 'Series A'] 
    return i 











In [69]:
train = clean(train)
val = clean(val)
test = clean(test)


In [70]:
train['Raised Series A'].value_counts()

No     12311
Yes      182
Name: Raised Series A, dtype: int64

In [71]:
train.shape , val.shape, test.shape

((12493, 20), (4576, 20), (4535, 20))

In [72]:
def column_name_change(p):
    p = p.copy()
    p= p.rename(columns = {'Organization Description':'Total Funding Amount','Organization Industries':'Total Funding Amount Currency',
    'Organization Location':'Total Funding Amount Currency (in USD)','Total Funding Amount':'Organization Description',
    'Total Funding Amount Currency':'Organization Industries','Total Funding Amount Currency (in USD)':'Organization Location',
    'Investor Names':'Number of Investors','Number of Investors':'Investor Names'})
    return p 

In [73]:
#change column names 
train = column_name_change(train)
val = column_name_change(val)
test = column_name_change(test)

In [74]:
train.sample(5)

,Transaction Name,Transaction Name URL,Organization Name,Organization Name URL,Funding Type,Money Raised,Money Raised Currency,Money Raised Currency (in USD),Announced Date,Total Funding Amount,Total Funding Amount Currency,Total Funding Amount Currency (in USD),Organization Description,Organization Industries,Organization Location,Number of Investors,Investor Names,Lead Investors,Lead Investors,Raised Series A
8514,Seed Round - Kapiva Ayurveda,https://www.crunchbase.com/funding_round/kapiv...,Kapiva Ayurveda,https://www.crunchbase.com/organization/kapiva...,Seed,2500000,USD,2500000.0,2019-01-25,2500000,USD,2500000,Kapiva Ayurveda is a fast-growing Ayurvedic fo...,"E-Commerce, Retail",NaN,1,Fireside Ventures,3,"Fireside Ventures, GITS Foods, Madhu Kela",No
5934,Seed Round - Bitgenia,https://www.crunchbase.com/funding_round/bitge...,Bitgenia,https://www.crunchbase.com/organization/bitgenia,Seed,NaN,NaN,NaN,2019-04-30,NaN,NaN,NaN,Bitgenia is an argentine bioinformatics compan...,"Bioinformatics, Health Care",NaN,1,NaN,1,StartUp Health,No
9250,Seed Round - Watsan Envirotech,https://www.crunchbase.com/funding_round/watsa...,Watsan Envirotech,https://www.crunchbase.com/organization/watsan...,Seed,NaN,NaN,NaN,2019-01-01,NaN,NaN,NaN,Watsan Envirotech is a manufacturing company.,"Manufacturing, Waste Management, Water Purific...",NaN,1,AIM Smart City,1,AIM Smart City,No
8484,Seed Round - RevenYOU - BOTS,https://www.crunchbase.com/funding_round/reven...,RevenYOU - BOTS,https://www.crunchbase.com/organization/revenyou,Seed,1800000,EUR,2036302.0,2019-01-24,5038000,EUR,5651225,RevenYOU gives the world access to automated t...,NaN,NaN,3,NaN,NaN,NaN,No
20217,Seed Round - Picture My Life,https://www.crunchbase.com/funding_round/pictu...,Picture My Life,https://www.crunchbase.com/organization/pictur...,Seed,196681,EUR,244031.0,2018-02-16,765396,USD,765396,Image-based communication tools that make life...,"Apps, Computer, iOS, Mobile, Software",NaN,4,Sciety,1,Sciety,No


In [77]:
def nlp_clean(z):
    z['Organization Description'] = z['Organization Description'].apply(lambda x: x.lower())
    z['Organization Industries'] = z['Organization Industries'].apply(lambda x: x.lower())
    z['Organiztion Location'] = z['Organization Location'].apply(lambda x: x.lower())
    z['Investor Names'] = z['Investor Names'].apply(lambda x: x.lower())
    z['Lead Investors'] = z['Lead Investors'].apply(lambda x: x.lower())
    return z 
